<a href="https://colab.research.google.com/github/kyugorithm/TIL/blob/main/study_CAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코랩 장시간 세션 유지 (https://naenjun.tistory.com/18)

In [ ]:
import os
from google.colab import drive 
!nvidia-smi
drive.mount('/content/gdrive/')

작업경로 pytorch-CycleGAN-Unet-D 로 변경

In [ ]:
os.chdir('/content/gdrive/MyDrive/')
root_dir = os.getcwd()
print(root_dir)

In [ ]:
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
transform = transforms.Compose([
                    transforms.Resize(224),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

])

In [ ]:
trainset = torchvision.datasets.STL10(root='./dataset', split = 'train', download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)
testset = torchvision.datasets.STL10(root='./dataset', split = 'test', download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
  img = img / 2 + 0.5
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg,(1,2,0)))

dataiter = iter(trainloader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))
# print(' '.join('%5s' % classes[labels[i] for i in range(4)]))

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv = nn.Sequential([
                               nn.Conv2d(3,    64, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.Conv2d(64,   64, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.MaxPool2d(2,2),
                               nn.Conv2d(64,  128, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.Conv2d(128, 128, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.MaxPool2d(2,2),
                               nn.Conv2d(128, 256, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.Conv2d(256, 256, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.MaxPool2d(2,2),
                               nn.Conv2d(256, 512, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.Conv2d(512, 512, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.Conv2d(512, 512, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.MaxPool2d(2,2),
                               nn.Conv2d(512, 512, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.Conv2d(512, 512, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.Conv2d(512, 512, 3, padding=1), nn.LeakyReLU(0.2),
                               nn.MaxPool2d(2,2),

    ])

    self.avg_pool = nn.AvgPool2d(7)
    self.classifier = nn.Linear(512,10)
    self.fc1 = nn.Linear(512*2*2, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, 10)

  def forward(self, x):
    